In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from vessel_manoeuvring_models.visualization.plot import track_plots, track_plot, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
#plt.style.use('paper')
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('pdf')

from src.symbols import *

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.prime_system import PrimeSystem

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline
from src.parameters import df_parameters
p = df_parameters['symbol']
from src.feature_selection import drop_multicollinearity
from src.bias_variance_tradeoff import train_test_split_run
from sklearn.metrics import r2_score
from wPCC_pipeline.pipelines.motion_regression.nodes import fit_motions, create_model_from_motion_regression
from src.models.regression import Regression
from myst_nb import glue
import src.symbols as s
from IPython.display import Latex
from src.substitute_dynamic_symbols import run
import seaborn as sns
from src.feature_selection import feature_imporance, BestFeatures, DropCorrelation
from wPCC_pipeline.turning_circle import TurningCircle
#from jb_helpers import parameter_to_latex, df_to_myst
from wPCC_pipeline.monte_carlo import monte_carlo, parameter_variation
from wPCC_pipeline.presentation import set_file_name, snapshot

In [ ]:
ship="wpcc"
#vmm_name = "vmm_martins_simple"
vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)

ship_data = catalog.load(f"{ship}.ship_data")

#regression = catalog.load(f"{ship}.updated.{vmm_name}.joined.regression")
#regression.diff_eq_Y.exclude_parameters.pop('Ydelta')
#regression.diff_eq_Y.exclude_parameters.pop('Ythrustdelta')

id = 22772
data_lowpass = catalog.load(f"{ ship }.{ id }.data_lowpass")

data_ekf_1 = catalog.load(f"{ ship }.initial.{ id }.data_ek_filter")
data_ekf_2 = catalog.load(f"{ ship }.updated.{ id }.data_ek_filter")
data_rts_2 = catalog.load(f"{ ship }.updated.{ id }.data_ek_smooth")

In [ ]:
ship_data

In [ ]:
track_plot(data_lowpass, lpp=ship_data['L'], beam=ship_data['B'])

In [ ]:
dataframes = {'Model test (LPF)':data_lowpass,
             'EKF 1':data_ekf_1,
             'EKF 2':data_ekf_2,
             'RTS 2':data_rts_2}

plot(dataframes, keys=['u','v','r','u1d','v1d','r1d']);

